(Updated Notebook)

# Install Packages and Setup Variables

In [1]:
!pip install -q llama-index==0.12.2 openai==1.70.0 llama-index-llms-gemini==0.4.1 tiktoken==0.8.0 chromadb==0.6.0
!pip install -q llama-index-vector-stores-chroma==0.4.1 llama-index-finetuning==0.3.1 llama-index-llms-together==0.3.1
!pip install -q llama-index-embeddings-huggingface==0.4.0 llama-index-readers-web==0.3.1 html2text==2024.2.26
!pip install -q sentence-transformers==3.3.1 pydantic==2.10.4 pandas==2.2.2 # kaleido==0.2.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.1/599.1 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 602.1/602.1 kB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 67.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 66.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 100.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 52.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 2.2 MB/s eta 0:00:00


In [2]:
# !pip install -q openai==1.70.0 chromadb==0.6.0 google-generativeai==0.8.4 langchain==0.3.25 langchain-chroma==0.2.3

# langchain-openai==0.3.16 langchain_google_genai==2.0.9 llama-index-llms-gemini==0.4.1 llama-index==0.12.21

# llama-index-vector-stores-chroma==0.4.1

In [3]:
import os

# Set environment variables for the API keys
os.environ["OPENAI_API_KEY"] = "<YOUR_API_KEY>"
os.environ["TOGETHER_AI_API_TOKEN"] = "<YOUR_API_KEY>"
os.environ["GOOGLE_API_KEY"] = "<YOUR_API_KEY>"

# from google.colab import userdata
# os.environ["OPENAI_API_KEY"] = userdata.get('openai_api_key')
# os.environ["GOOGLE_API_KEY"] = userdata.get('Google_api_key')
# os.environ["TOGETHER_AI_API_TOKEN"] = userdata.get('together_api_key')


# Allows running asyncio in environments with an existing event loop, like Jupyter notebooks.
import nest_asyncio

nest_asyncio.apply()

# Create a vector store and ingest articles

In [4]:
import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore

# create vector store
vector_store_name = "mini-llama-articles"
chroma_client = chromadb.PersistentClient(path=vector_store_name)
chroma_collection = chroma_client.get_or_create_collection(vector_store_name)
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

The dataset includes several articles from the TowardsAI blog, which provide an in-depth explanation of the LLaMA2 model. Read the dataset as a long string.

In [5]:
!curl -o ./mini-llama-articles.csv https://raw.githubusercontent.com/AlaFalaki/tutorial_notebooks/main/data/mini-llama-articles.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  169k  100  169k    0     0   426k      0 --:--:-- --:--:-- --:--:--  426k


## Read articles from file

In [6]:
import csv

rows = []

# Load the file as a JSON
with open("./mini-llama-articles.csv", mode="r", encoding="utf-8") as file:
  csv_reader = csv.reader(file)

  for idx, row in enumerate( csv_reader ):
    if idx == 0: continue; # Skip header row
    rows.append(row)

## Ingest documents into vector store

In [7]:
from llama_index.core import Document
from llama_index.core.text_splitter import TokenTextSplitter
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.ingestion import IngestionPipeline

# Convert the chunks to Document objects so the LlamaIndex framework can process them.
documents = [Document(text=row[1], metadata={"title": row[0], "url": row[2], "source_name": row[3]}) for row in rows]

# Define the splitter object that split the text into segments with 512 tokens,
# with a 128 overlap between the segments.
text_splitter = TokenTextSplitter(
    separator=" ", chunk_size=512, chunk_overlap=128
)

# Create the pipeline to apply the transformation on each chunk,
# and store the transformed text in the chroma vector store.
pipeline = IngestionPipeline(
    transformations=[
        text_splitter,
        HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5") # Or, OpenAIEmbedding()
    ],
    vector_store=vector_store
)

nodes = pipeline.run(documents=documents, show_progress=True)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Parsing nodes:   0%|          | 0/14 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/108 [00:00<?, ?it/s]

# Load vector store and create query engine

In [8]:
from llama_index.core.prompts import PromptTemplate
from llama_index.llms.together import TogetherLLM
from llama_index.core import VectorStoreIndex

# Use the Together AI service to access the LLaMA2-70B chat model
llm = TogetherLLM(
    model="meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo",
    api_key=os.environ["TOGETHER_AI_API_TOKEN"]
)

# create index from vector store
index = VectorStoreIndex.from_vector_store(vector_store, embed_model="local:BAAI/bge-small-en-v1.5")

# Define a query engine that is responsible for retrieving related pieces of text,
# and using a LLM to formulate the final answer.
query_engine = index.as_query_engine(llm=llm)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# Test query engine

In [9]:
res = query_engine.query("How many parameters LLaMA2 has?")
print(res.response)
# 'Llama 2 is available in four different model sizes: 7 billion, 13 billion, 34 billion, and 70 billion parameters.'

Llama 2 is available in four different model sizes: 7 billion, 13 billion, 34 billion, and 70 billion parameters.


In [10]:
# print the source nodes used to write the answer
for src in res.source_nodes:
  print("Node ID\t", src.node_id)
  print("Title\t", src.metadata['title'])
  print("Text\t", src.text)
  print("Score\t", src.score)
  print("-_"*20)

Node ID	 f760111a-4acf-434a-b289-35c7b7de94dd
Title	 Meta's Llama 2: Revolutionizing Open Source Language Models for Commercial Use
Text	 I. Llama 2: Revolutionizing Commercial Use Unlike its predecessor Llama 1, which was limited to research use, Llama 2 represents a major advancement as an open-source commercial model. Businesses can now integrate Llama 2 into products to create AI-powered applications. Availability on Azure and AWS facilitates fine-tuning and adoption. However, restrictions apply to prevent exploitation. Companies with over 700 million active daily users cannot use Llama 2. Additionally, its output cannot be used to improve other language models.  II. Llama 2 Model Flavors Llama 2 is available in four different model sizes: 7 billion, 13 billion, 34 billion, and 70 billion parameters. While 7B, 13B, and 70B have already been released, the 34B model is still awaited. The pretrained variant, trained on a whopping 2 trillion tokens, boasts a context window of 4096 toke

# Evaluate the retriever

In [11]:
from llama_index.core.evaluation import generate_question_context_pairs
from llama_index.llms.openai import OpenAI

# Create questions for each segment. These questions will be used to
# assess whether the retriever can accurately identify and return the
# corresponding segment when queried.
llm = OpenAI(model="gpt-4o-mini")
rag_eval_dataset = generate_question_context_pairs(
    nodes,
    llm=llm,
    num_questions_per_chunk=1
)

# We can save the evaluation dataset as a json file for later use.
rag_eval_dataset.save_json("./rag_eval_dataset.json")

100%|██████████| 108/108 [01:59<00:00,  1.11s/it]


If you have uploaded the generated question JSON file, please uncomment the code in the next cell block. This will avoid the need to generate the questions manually, saving you time and effort.

In [12]:
#from llama_index.finetuning.embeddings.common import EmbeddingQAFinetuneDataset

from llama_index.core.evaluation import EmbeddingQAFinetuneDataset

rag_eval_dataset = EmbeddingQAFinetuneDataset.from_json(
    "./rag_eval_dataset.json"
)

In [13]:
import asyncio
from llama_index.core.evaluation import RetrieverEvaluator, RelevancyEvaluator, FaithfulnessEvaluator, BatchEvalRunner
from llama_index.llms.openai import OpenAI

async def run_evaluation(index, rag_eval_dataset, top_k_values, llm_judge,llm, n_queries_to_evaluate=20,num_work=1):
    evaluation_results = {}

    # ------------------- MRR and Hit Rate -------------------

    for top_k in top_k_values:
        # Get MRR and Hit Rate
        retriever = index.as_retriever(similarity_top_k=top_k)
        retriever_evaluator = RetrieverEvaluator.from_metric_names(
            ["mrr", "hit_rate"], retriever=retriever
        )
        eval_results = await retriever_evaluator.aevaluate_dataset(rag_eval_dataset)
        avg_mrr = sum(res.metric_vals_dict["mrr"] for res in eval_results) / len(eval_results)
        avg_hit_rate = sum(res.metric_vals_dict["hit_rate"] for res in eval_results) / len(eval_results)

        # Collect the evaluation results
        evaluation_results[f"mrr_@_{top_k}"] = avg_mrr
        evaluation_results[f"hit_rate_@_{top_k}"] = avg_hit_rate

    # ------------------- Faithfulness and Relevancy -------------------

    # Extract the questions from the dataset
    queries = list(rag_eval_dataset.queries.values())
    batch_eval_queries = queries[:n_queries_to_evaluate]

    # Initiate the faithfulnes and relevancy evaluator objects
    faithfulness_evaluator = FaithfulnessEvaluator(llm=llm_judge)
    relevancy_evaluator = RelevancyEvaluator(llm=llm_judge)

    # The batch evaluator runs the evaluation in batches
    runner = BatchEvalRunner(
        {
            "faithfulness": faithfulness_evaluator,
            "relevancy": relevancy_evaluator
        },
        workers=num_work,
        show_progress=True,
    )

    # Get faithfulness and relevancy scores
    query_engine = index.as_query_engine(llm=llm)
    eval_results = await runner.aevaluate_queries(
        query_engine, queries=batch_eval_queries
    )
    faithfulness_score = sum(result.passing for result in eval_results['faithfulness']) / len(eval_results['faithfulness'])
    relevancy_score = sum(result.passing for result in eval_results['relevancy']) / len(eval_results['relevancy'])
    evaluation_results["faithfulness"] = faithfulness_score
    evaluation_results["relevancy"] = relevancy_score

    return evaluation_results

In [14]:
# We evaluate the retrievers with different top_k values.
top_k_values = [2, 4, 6, 8, 10]

llm_judge = OpenAI(temperature=0, model="gpt-4o")

evaluation_results = await run_evaluation(index, rag_eval_dataset, top_k_values, llm_judge,llm=llm,n_queries_to_evaluate=20,num_work=1)

100%|██████████| 40/40 [00:40<00:00,  1.02s/it]


In [15]:
print(evaluation_results)

{'mrr_@_2': 0.7870370370370371, 'hit_rate_@_2': 0.8703703703703703, 'mrr_@_4': 0.8016975308641976, 'hit_rate_@_4': 0.9259259259259259, 'mrr_@_6': 0.8050925925925926, 'hit_rate_@_6': 0.9444444444444444, 'mrr_@_8': 0.8077380952380954, 'hit_rate_@_8': 0.9629629629629629, 'mrr_@_10': 0.8077380952380954, 'hit_rate_@_10': 0.9629629629629629, 'faithfulness': 1.0, 'relevancy': 1.0}


In [16]:
# Use GPT-4o-mini as the LLM model
llm = OpenAI(model="gpt-4o-mini")

# run evaluation with GPT-4o-mini
top_k_values = [2, 4, 6, 8, 10]
llm_judge = OpenAI(temperature=0, model="gpt-4o")
evaluation_results = await run_evaluation(index, rag_eval_dataset, top_k_values, llm_judge,llm=llm, n_queries_to_evaluate=20,num_work=16)

100%|██████████| 40/40 [00:06<00:00,  6.30it/s]


In [17]:
print(evaluation_results)

{'mrr_@_2': 0.7870370370370371, 'hit_rate_@_2': 0.8703703703703703, 'mrr_@_4': 0.8016975308641976, 'hit_rate_@_4': 0.9259259259259259, 'mrr_@_6': 0.8050925925925926, 'hit_rate_@_6': 0.9444444444444444, 'mrr_@_8': 0.8077380952380954, 'hit_rate_@_8': 0.9629629629629629, 'mrr_@_10': 0.8077380952380954, 'hit_rate_@_10': 0.9629629629629629, 'faithfulness': 1.0, 'relevancy': 1.0}


In [18]:
# from llama_index.llms.gemini import Gemini

# # Use Gemini as the LLM model
# llm = Gemini(model="models/gemini-1.5-flash")

# # run evaluation with Gemini
# top_k_values = [2, 4, 6, 8, 10]
# llm_judge = OpenAI(temperature=0, model="gpt-4o")
# evaluation_results = await run_evaluation(index, rag_eval_dataset, top_k_values, llm_judge,llm=llm, n_queries_to_evaluate=20,num_work=1)

# Inference speed comparison

In [19]:
import time

In [20]:
llm = TogetherLLM(
    model="meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo",
    api_key=os.environ["TOGETHER_AI_API_TOKEN"]
)

time_start = time.time()
llm.complete("List the 50 states in the United States of America. Write their names in a comma-separated list and nothing else.")
time_end = time.time()
print("Time taken for Llama 3.1 70B on Together AI: {0:.2f} seconds".format(time_end - time_start))

Time taken for Llama 3.1 70B on Together AI: 2.44 seconds


In [21]:
llm = OpenAI(model="gpt-4o-mini")

time_start = time.time()
llm.complete("List the 50 states in the United States of America. Write their names in a comma-separated list and nothing else.")
time_end = time.time()
print("Time taken for GPT 4o Mini: {0:.2f} seconds".format(time_end - time_start))

Time taken for GPT 4o Mini: 2.80 seconds


In [22]:
from llama_index.llms.gemini import Gemini

llm = Gemini(model="models/gemini-1.5-flash")

time_start = time.time()
llm.complete("List the 50 states in the United States of America. Write their names in a comma-separated list and nothing else.")
time_end = time.time()
print("Time taken for Gemini 1.5 Flash: {0:.2f} seconds".format(time_end - time_start))

Time taken for Gemini 1.5 Flash: 2.69 seconds
